<a href="https://colab.research.google.com/github/sohamtalukdar/Enron-Email-Analysis/blob/main/Vlabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/MyDrive/maildir/"
!tar -xzvf "/content/drive/MyDrive/enron_with_categories.tar.gz" -C "/content/drive/MyDrive/maildir/"

In [ ]:
import csv

with open("/content/drive/MyDrive/maildir/enron_with_categories/1/173998.txt") as f:
    data = f.read()

fields = ["Message-ID", "Date", "From", "To", "Subject", "Cc", "Mime-Version", "Content-Type", "Content-Transfer-Encoding", "Bcc", "X-From", "X-To", "X-cc", "X-bcc", "X-Folder", "X-Origin", "X-FileName", "Message Body"]
rows = data.split("\n")
email_dict = {}
message_body = ""
for row in rows:
    for field in fields:
        if field in row:
            email_dict[field] = row.split(": ")[1]
    if not any(field in row for field in fields):
        message_body += row + "\n"

email_dict["Message Body"] = message_body

with open("email.csv", "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=fields)
    writer.writeheader()
    writer.writerow(email_dict)


In [ ]:
import pandas as pd 
pd.read_csv("/content/email.csv")

,Message-ID,Date,From,To,Subject,Cc,Mime-Version,Content-Type,Content-Transfer-Encoding,Bcc,X-From,X-To,X-cc,X-bcc,X-Folder,X-Origin,X-FileName,Message Body
0,<16727181.1075846164914.JavaMail.evans@thyme>,"Tue, 8 Aug 2000 05:54:00 -0700 (PDT)",Steven J Kean,"""'Adhoc Advisors'"" <Rhonda_Roo_00@yahoo.com>",More great press!,richard.shapiro@enron.com,1.0,text/plain; charset=us-ascii,7bit,richard.shapiro@enron.com,Steven J Kean,GFoster@Antigenics.com,Richard Shapiro,NaN,\Steven_Kean_Dec2000_1\Notes Folders\All docum...,KEAN-S,skean.nsf,\nDid you see Peggy Noonan's article? Shapiro...
